# 111

In [1]:
# Sanity check of your system and your brain :)
"A" == "A"

True

In [2]:
# for reproducibility
import numpy as np
import random
random_seed=42
np.random.seed(random_seed)
random.seed(random_seed)


## Preparing data and black (In Orange framework, it is simple and good)

In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np

from prepare_dataset import prepare_adult_dataset,prepare_german_dataset,prepare_compas_dataset,prepare_pima_dataset,train_test_split_data,prepare_2d_sinusoidal_dataset
from utils import encoder_from_datatable


data_table,test_data_table = prepare_adult_dataset()
# data_table,test_data_table = prepare_2d_sinusoidal_dataset()

# data_table,test_data_table = prepare_german_dataset()

# data_table,test_data_table = prepare_compas_dataset()

# data_table, vali_data_table = train_test_split_data(data_table)
# data_table,test_data_table = prepare_pima_dataset()

print("shape of train set", data_table.X.shape)
# print("instances of evaluation set", vali_data_table.X.shape[0])
print("shape of test set", test_data_table.X.shape)


/Users/lucy/Desktop/workspace/Descriptive-Induction-ML/prepare_dataset.py:153: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if '?' in df[col].unique():
/Users/lucy/Desktop/workspace/Descriptive-Induction-ML/prepare_dataset.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col][df[col] == '?'] = df[col].value_counts().index[0]


shape of train set (43957, 12)
shape of test set (4885, 12)


In [4]:

from prepare_blackbox import train_classifier

black_box_model = train_classifier(data_table,classifier_method='dnn',random_seed=random_seed)
# black_box_model = train_classifier(data_table,classifier_method='rf',random_seed=random_seed)



black_box = lambda x: black_box_model.predict(x)
# black_box = lambda x:  c.predict(scikit_encoder.transform(x))

print("now start sanity-check")
print("the first row of the dataset is (orginal form):\n", data_table[0] )
print("the predition of blackbox on the first row:\n", black_box( [data_table.X[0]]  )   )

print("train set data set size,",data_table.X.shape)
print("test setdata set size,",test_data_table.X.shape)
print("data set size,",data_table[:5])

now start sanity-check
the first row of the dataset is (orginal form):
 [20, Private, Some-college, Never-married, Other-service, Own-child, White, Female, 0, 0, 32, United-States | <=50K]
the predition of blackbox on the first row:
 [0.]
train set data set size, (43957, 12)
test setdata set size, (4885, 12)
data set size, [[20, Private, Some-college, Never-married, Other-service, Own-child, White, Female, 0, 0, 32, United-States | <=50K],
 [47, Private, Bachelors, Never-married, Sales, Own-child, White, Male, 0, 0, 40, United-States | <=50K],
 [35, Local-gov, HS-grad, Never-married, Adm-clerical, Unmarried, Amer-Indian-Eskimo, Male, 0, 0, 55, United-States | <=50K],
 [30, Private, Bachelors, Never-married, Sales, Own-child, White, Male, 0, 0, 40, United-States | >50K],
 [31, Self-emp-not-inc, Assoc-acdm, Married-civ-spouse, Other-service, Wife, White, Female, 0, 0, 35, United-States | >50K]]


A sanity check of train and test accuracy

In [5]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
import sklearn
# predict_fn = lambda x: aml.predict(H2OFrame(x)).as_data_frame().iloc[:,0].to_numpy()

predict_fn = black_box

print('Train acc', accuracy_score(data_table.Y, predict_fn(data_table.X)))
print('Test acc', accuracy_score(test_data_table.Y, predict_fn(test_data_table.X)))

print('acc on training set', sklearn.metrics.accuracy_score(data_table.Y, predict_fn(data_table.X)))
print('f1 on training set', sklearn.metrics.f1_score(data_table.Y, predict_fn(data_table.X)))
print('recall on training set', sklearn.metrics.recall_score(data_table.Y, predict_fn(data_table.X)))
print('precision on training set', sklearn.metrics.precision_score(data_table.Y, predict_fn(data_table.X)))

print('acc on test set', sklearn.metrics.accuracy_score(test_data_table.Y, predict_fn(test_data_table.X)))
print('f1 on test set', sklearn.metrics.f1_score(test_data_table.Y, predict_fn(test_data_table.X)))
print('recall on test set', sklearn.metrics.recall_score(test_data_table.Y, predict_fn(test_data_table.X)))
print('precision on test set', sklearn.metrics.precision_score(test_data_table.Y, predict_fn(test_data_table.X)))


# print('test f1', f1_score(y_test, predict_fn(X_test)))
# print('test recall',recall_score(y_test, predict_fn(X_test)))
# print('test precision', precision_score(y_test, predict_fn(X_test)))


Train acc 0.9280660645630957
Test acc 0.8354145342886387
acc on training set 0.9280660645630957
f1 on training set 0.8416149068322982
recall on training set 0.7989538754160723
precision on training set 0.8890887924648111
acc on test set 0.8354145342886387
f1 on test set 0.6375112714156899
recall on test set 0.6032423208191127
precision on test set 0.6759082217973231


In [6]:
black_box = predict_fn

from utils import label_with_blackbox

predicted_data_table = label_with_blackbox(data_table,black_box)

predicted_test_data_table = label_with_blackbox(test_data_table,black_box)


print(predicted_data_table[:5])
print(predicted_data_table.X[:5])
print(predicted_data_table.X.shape[0])
print(predicted_test_data_table.X.shape[0])

print('sanity check, of acc,should be 100', accuracy_score(predicted_data_table.Y, predict_fn(predicted_data_table.X)))

[[20, Private, Some-college, Never-married, Other-service, Own-child, White, Female, 0, 0, 32, United-States | <=50K],
 [47, Private, Bachelors, Never-married, Sales, Own-child, White, Male, 0, 0, 40, United-States | <=50K],
 [35, Local-gov, HS-grad, Never-married, Adm-clerical, Unmarried, Amer-Indian-Eskimo, Male, 0, 0, 55, United-States | <=50K],
 [30, Private, Bachelors, Never-married, Sales, Own-child, White, Male, 0, 0, 40, United-States | <=50K],
 [31, Self-emp-not-inc, Assoc-acdm, Married-civ-spouse, Other-service, Wife, White, Female, 0, 0, 35, United-States | >50K]]
[[20.  3. 15.  4.  7.  3.  4.  0.  0.  0. 32. 38.]
 [47.  3.  9.  4. 11.  3.  4.  1.  0.  0. 40. 38.]
 [35.  1. 11.  4.  0.  4.  0.  1.  0.  0. 55. 38.]
 [30.  3.  9.  4. 11.  3.  4.  1.  0.  0. 40. 38.]
 [31.  5.  7.  2.  7.  5.  4.  0.  0.  0. 35. 38.]]
43957
4885
sanity check, of acc,should be 100 1.0


## Run once. Our approach Non-active mode ($\beta=zero$ )

In [47]:
%load_ext autoreload
%autoreload 2
from approach import explain_tabular
l_parameter = 0.0001
# l_parameter = 0.0005
explanations,ADS = explain_tabular(predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed,beta=0,lambda_parameter=l_parameter )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
init transformer okay!
new cached!


ValueError: max() arg is an empty sequence

In [40]:
%matplotlib inline

from utils import  rule_to_string,ruleset_predict
# ADS.finish()
# explanations = ADS.output_the_best(0.0003)
for e in explanations:
    print(rule_to_string(e,data_table.domain,target_class_idx=1))
    
our_prediction = ruleset_predict(explanations,test_data_table.X)

print(sum(our_prediction) )

import sklearn
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

from utils import  rule_to_string
print('number of rules:',len(explanations))
print("new instances",ADS.synthetic_data_table.X.shape[0])

from utils import compute_metrics

compute_metrics(explanations,data_table.domain)


IF 12614.0 <= capital-gain <= 99999.0 THEN income=>50K 
IF 1894.5 <= capital-loss <= 1978.5 THEN income=>50K 
IF 5178.0 <= capital-gain <= 99999.0 THEN income=>50K 
IF 7280.0 <= capital-gain <= 9999.900000000001 THEN income=>50K 
IF education is Bachelors,Doctorate AND marital-status is Married-civ-spouse AND occupation is Adm-clerical,Craft-repair,Prof-specialty,Sales,Tech-support,Transport-moving THEN income=>50K 
IF education is Doctorate THEN income=>50K 
IF education is Masters AND marital-status is Married-civ-spouse THEN income=>50K 
IF education is Masters AND native-country is Italy THEN income=>50K 
IF education is Prof-school AND marital-status is Married-civ-spouse THEN income=>50K 
IF native-country is Greece THEN income=>50K 
IF native-country is Taiwan THEN income=>50K 
IF native-country is Yugoslavia THEN income=>50K 
IF relationship is Other-relative AND 4999.950000000001 <= capital-gain <= 99999.0 THEN income=>50K 
IF workclass is Federal-gov,Private AND occupation is

In [41]:
lambda_candidates = [  0,0.0000001,0.000001,0.00001,0.00003,0.00005,0.00008,0.0001,0.0005,0.001,0.0013,0.0015,0.0018,0.002,0.003,0.004,0.0045,0.005,0.006,0.007,0.008,0.01,0.015,0.02,0.03,0.05,0.1,0.15,0.2,0.5 ]

beta = 0
for lambda_parameter in  lambda_candidates:
    explanations = ADS.output_the_best(lambda_parameter)
    our_prediction = ruleset_predict(explanations,test_data_table.X)
    f1_score = sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction)
    acc_score=sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction)
    rec_score = sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction)
    pre_score = sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction)
    num = len(explanations)
    num_of_instance = ADS.synthetic_data_table.X.shape[0]

    print("parameter: ",beta,lambda_parameter,"metrics",num,f1_score,acc_score,rec_score,pre_score,num_of_instance)

best solution found in iteration 644
parameter:  0 0 metrics 16 0.7322953289804119 0.8908904810644831 0.6969407265774379 0.7714285714285715 0
best solution found in iteration 644
parameter:  0 1e-07 metrics 16 0.7322953289804119 0.8908904810644831 0.6969407265774379 0.7714285714285715 0
best solution found in iteration 644
parameter:  0 1e-06 metrics 16 0.7322953289804119 0.8908904810644831 0.6969407265774379 0.7714285714285715 0
best solution found in iteration 644
parameter:  0 1e-05 metrics 16 0.7322953289804119 0.8908904810644831 0.6969407265774379 0.7714285714285715 0
best solution found in iteration 644
parameter:  0 3e-05 metrics 16 0.7322953289804119 0.8908904810644831 0.6969407265774379 0.7714285714285715 0
best solution found in iteration 644
parameter:  0 5e-05 metrics 16 0.7322953289804119 0.8908904810644831 0.6969407265774379 0.7714285714285715 0
best solution found in iteration 666
parameter:  0 8e-05 metrics 14 0.7039337474120082 0.882906857727738 0.6500956022944551 0.76

## Run once. Our approach active mode

In [42]:
%load_ext autoreload
%autoreload 2
from approach import explain_tabular
# beta = 1.2e-04
beta = 5e-05
# beta = 1.5e-04
l_parameter = 0.0001

explanations_active,ADS_active = explain_tabular(predicted_data_table, black_box, target_class_idx=1, random_seed=42,beta=beta,lambda_parameter=l_parameter)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
init transformer okay!
new cached!


best solution found in iteration 601
Now print Error Log
at iteration: 4 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 5 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 6 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 7 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 33 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 42 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 52 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at

In [43]:

from utils import  rule_to_string,ruleset_predict
# explanations_active = ADS_active.output_the_best(0.0001)
print("num of rules",len(explanations_active))
for e in explanations_active:
    print(rule_to_string(e,data_table.domain,target_class_idx=1))
    
our_prediction = ruleset_predict(explanations_active,test_data_table.X)

print(sum(our_prediction) )

from utils import label_with_blackbox
predicted_test_data_table = label_with_blackbox(test_data_table,black_box)
print(sum(predicted_test_data_table.Y) )
print("number of rules",len(explanations_active))
import sklearn
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

from utils import compute_metrics

compute_metrics(explanations_active,data_table.domain)

num of rules 10
IF 19999.800000000003 <= capital-gain <= 99999.0 AND 15.700000000000003 <= hours-per-week <= 99.0 THEN income=>50K 
IF 27.950000000000003 <= age <= 90.0 AND 9999.900000000001 <= capital-gain <= 99999.0 THEN income=>50K 
IF 38.900000000000006 <= age <= 53.5 AND workclass is Federal-gov,Local-gov,Self-emp-inc AND education is Assoc-acdm,Assoc-voc,Some-college AND marital-status is Married-civ-spouse THEN income=>50K 
IF 7280.0 <= capital-gain <= 12614.0 AND native-country is Philippines,United-States THEN income=>50K 
IF education is Bachelors,Doctorate AND relationship is Husband,Wife AND 35.300000000000004 <= hours-per-week <= 74.5 THEN income=>50K 
IF education is Masters,Prof-school AND marital-status is Married-civ-spouse AND native-country is United-States THEN income=>50K 
IF education is Prof-school AND 50.5 <= hours-per-week <= 99.0 THEN income=>50K 
IF marital-status is Married-civ-spouse AND 1742.4 <= capital-loss <= 4356.0 THEN income=>50K 
IF marital-status i

In [44]:
lambda_candidates = [  0,0.0000001,0.000001,0.00001,0.00003,0.00005,0.00008,0.0001,0.0005,0.001,0.0013,0.0015,0.0018,0.002,0.0025,0.003,0.005,0.006,0.007,0.008,0.01,0.015,0.02,0.03,0.05,0.1,0.15,0.2,0.5 ]


for lambda_parameter in  lambda_candidates:
    explanations = ADS_active.output_the_best(lambda_parameter)
    our_prediction = ruleset_predict(explanations,test_data_table.X)
    f1_score = sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction)
    acc_score=sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction)
    rec_score = sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction)
    pre_score = sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction)
    num = len(explanations)
    num_of_instance = ADS_active.synthetic_data_table.X.shape[0]

    print("parameter: ",beta,lambda_parameter,"metrics",num,f1_score,acc_score,rec_score,pre_score,num_of_instance)

best solution found in iteration 586
parameter:  5e-05 0 metrics 11 0.7313357034027427 0.8917093142272262 0.6883365200764818 0.7800650054171181 14766
best solution found in iteration 586
parameter:  5e-05 1e-07 metrics 11 0.7313357034027427 0.8917093142272262 0.6883365200764818 0.7800650054171181 14766
best solution found in iteration 586
parameter:  5e-05 1e-06 metrics 11 0.7313357034027427 0.8917093142272262 0.6883365200764818 0.7800650054171181 14766
best solution found in iteration 586
parameter:  5e-05 1e-05 metrics 11 0.7313357034027427 0.8917093142272262 0.6883365200764818 0.7800650054171181 14766
best solution found in iteration 586
parameter:  5e-05 3e-05 metrics 11 0.7313357034027427 0.8917093142272262 0.6883365200764818 0.7800650054171181 14766
best solution found in iteration 586
parameter:  5e-05 5e-05 metrics 11 0.7313357034027427 0.8917093142272262 0.6883365200764818 0.7800650054171181 14766
best solution found in iteration 586
parameter:  5e-05 8e-05 metrics 11 0.731335

In [45]:
%load_ext autoreload
%autoreload 2
from approach import explain_tabular
# beta = 1.2e-04
beta = 1e-4
# beta = 1.5e-04
l_parameter = 0.0001

explanations_active,ADS_active = explain_tabular(predicted_data_table, black_box, target_class_idx=1, random_seed=42,beta=beta,lambda_parameter=l_parameter)

from utils import  rule_to_string,ruleset_predict
# explanations_active = ADS_active.output_the_best(0.0001)
print("num of rules",len(explanations_active))
for e in explanations_active:
    print(rule_to_string(e,data_table.domain,target_class_idx=1))
    
our_prediction = ruleset_predict(explanations_active,test_data_table.X)

print(sum(our_prediction) )

from utils import label_with_blackbox
predicted_test_data_table = label_with_blackbox(test_data_table,black_box)
print(sum(predicted_test_data_table.Y) )
print("number of rules",len(explanations_active))
import sklearn
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

from utils import compute_metrics

compute_metrics(explanations_active,data_table.domain)

lambda_candidates = [  0,0.0000001,0.000001,0.00001,0.00003,0.00005,0.00008,0.0001,0.0005,0.001,0.0013,0.0015,0.0018,0.002,0.0025,0.003,0.005,0.006,0.007,0.008,0.01,0.015,0.02,0.03,0.05,0.1,0.15,0.2,0.5 ]


for lambda_parameter in  lambda_candidates:
    explanations = ADS_active.output_the_best(lambda_parameter)
    our_prediction = ruleset_predict(explanations,test_data_table.X)
    f1_score = sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction)
    acc_score=sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction)
    rec_score = sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction)
    pre_score = sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction)
    num = len(explanations)
    num_of_instance = ADS_active.synthetic_data_table.X.shape[0]

    print("parameter: ",beta,lambda_parameter,"metrics",num,f1_score,acc_score,rec_score,pre_score,num_of_instance)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
init transformer okay!
new cached!


best solution found in iteration 772
Now print Error Log
at iteration: 4 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 5 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 6 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 7 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 33 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 364 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')
at iteration: 368 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'REMOVE_CONDITION')


best solution found in iteration 822
parameter:  0.0001 0.008 metrics 3 0.6866485013623979 0.8822927328556807 0.6022944550669216 0.7984790874524715 29416
best solution found in iteration 822
parameter:  0.0001 0.01 metrics 3 0.6866485013623979 0.8822927328556807 0.6022944550669216 0.7984790874524715 29416
best solution found in iteration 895
parameter:  0.0001 0.015 metrics 2 0.6694045174537987 0.8681678607983623 0.6233269598470363 0.7228381374722838 29416
best solution found in iteration 895
parameter:  0.0001 0.02 metrics 2 0.6694045174537987 0.8681678607983623 0.6233269598470363 0.7228381374722838 29416
best solution found in iteration 426
parameter:  0.0001 0.03 metrics 1 0.3493312352478364 0.8307062436028659 0.21223709369024857 0.9866666666666667 29416
best solution found in iteration 426
parameter:  0.0001 0.05 metrics 1 0.3493312352478364 0.8307062436028659 0.21223709369024857 0.9866666666666667 29416
best solution found in iteration 426
parameter:  0.0001 0.1 metrics 1 0.349331

## Baselines

### IDS

In [16]:
print("loadad")

loadad


In [17]:
%load_ext autoreload
%autoreload 2


import Orange
disc = Orange.preprocess.Discretize()
disc.method = Orange.preprocess.discretize.EntropyMDL()
# disc.method = Orange.preprocess.discretize.EqualFreq(n=3)

disc_predicted_data_table = disc(predicted_data_table)
# disc_predicted_test_data_table = disc(predicted_test_data_table)
disc_predicted_test_data_table = Orange.data.Table.from_table(disc_predicted_data_table.domain, predicted_test_data_table)

from competition_methods_explanation.passive_methods.ids import explain_tabular
iter_max = 1000
explanations = explain_tabular(disc_predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed)
# explanations = explain_tabular(data_table, black_box, target_class_idx=1, random_seed=random_seed)

from utils import  rule_to_string,ruleset_predict
our_prediction = ruleset_predict(explanations,disc_predicted_test_data_table.X)

import sklearn
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(disc_predicted_test_data_table.Y, our_prediction))

from competition_methods_explanation.passive_methods.ids import compute_metrics_ids
compute_metrics_ids(explanations)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
start Apriori
finish Apriori. Converting itemset
Pre-mined okay. all pre-mined rules of 9742
data set shape 43957
init compute


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm_notebook
%load_ext autoreload
%autoreload 2

import Orange

disc = Orange.preprocess.Discretize()
disc.method = Orange.preprocess.discretize.EntropyMDL()

disc_predicted_data_table = disc(predicted_data_table)
# disc_predicted_test_data_table = disc(predicted_test_data_table)
disc_predicted_test_data_table = Orange.data.Table.from_table(disc_predicted_data_table.domain, predicted_test_data_table)


from utils import uniform_enlarge_dataset,estimated_enlarge_dataset

rate = 1.0 * 49804 / predicted_data_table.X.shape[0] 
# rate = 1.13
print("sampling rate",rate)
new_predicted_data_table = estimated_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)
print(new_predicted_data_table.X.shape)
disc_new_predicted_data_table =  Orange.data.Table.from_table(disc_predicted_data_table.domain, new_predicted_data_table)


from competition_methods_explanation.passive_methods.ids import explain_tabular
explanations = explain_tabular(disc_new_predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed)

from utils import  rule_to_string,ruleset_predict
our_prediction = ruleset_predict(explanations,disc_predicted_test_data_table.X)

import sklearn
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(disc_predicted_test_data_table.Y, our_prediction))

from competition_methods_explanation.passive_methods.ids import compute_metrics_ids
compute_metrics_ids(explanations)


### BRS

In [7]:
np.any([[True, False], [False, False],[False, False]], axis=0)
# print("dasdas")

array([ True, False])

In [8]:
%load_ext autoreload
%autoreload 2


import Orange
disc = Orange.preprocess.Discretize()
disc.method = Orange.preprocess.discretize.EntropyMDL()
# disc.method = Orange.preprocess.discretize.EqualFreq(n=3)

disc_predicted_data_table = disc(predicted_data_table)
# disc_predicted_test_data_table = disc(predicted_test_data_table)
disc_predicted_test_data_table = Orange.data.Table.from_table(disc_predicted_data_table.domain, predicted_test_data_table)

# from competition_methods_explanation.passive_methods.brs import explain_tabular
from approach import explain_tabular
explanations,explainer = explain_tabular(disc_predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed,beta = 0,use_pre_mined=True, objective = 'bayesian')
# explanations = explain_tabular(data_table, black_box, target_class_idx=1, random_seed=random_seed)
# explanations,explainer = explain_tabular(disc_predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed,beta = 0,use_pre_mined=True, objective = 'simple')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
init transformer okay!
new cached!


using pre-mined rules
start Generating rule space.  Min support: 0.01
number of premined rules 46902
Screening rules using information gain
	Took 1957.627s to generate 2 rules

best solution found in iteration 775
Now print Error Log
at iteration: 2 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'ADD_RULE')
at iteration: 3 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'ADD_RULE')
at iteration: 4 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'ADD_RULE')
at iteration: 14 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'SPECIFY_CONDITION')
at iteration: 21 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'ADD_RULE')
at iteration: 22 happens the following error
('no plausible actions, len length of actions', 0, 'in the mode of ', 'ADD_RULE'

In [9]:

from utils import  rule_to_string,ruleset_predict
for e in explanations:
    try:
        print(rule_to_string(e,disc_predicted_data_table.domain,target_class_idx=1))
    except:
        continue

        
disc_predicted_test_data_table = disc_predicted_data_table
our_prediction = ruleset_predict(explanations,disc_predicted_test_data_table.X)

print( sum(our_prediction) )

import sklearn
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(disc_predicted_test_data_table.Y, our_prediction))

from competition_methods_explanation.passive_methods.brs import compute_metrics_brs
compute_metrics_brs(explanations)

IF capital-gain is 3048 - 3120,4243.5 - 4401,5119 - 5316.5,≥ 7055.5 THEN income=>50K 
IF capital-gain is 4668.5 - 4826 THEN income=>50K 
IF capital-loss is 1551.5 - 1568.5,1881.5 - 1927.5,1975.5 - 1978.5 THEN income=>50K 
IF capital-loss is 2364.5 - 3089.5 AND native-country is Poland,United-States THEN income=>50K 
IF education is Assoc-acdm,Bachelors,Doctorate,Masters,Prof-school AND marital-status is Married-civ-spouse AND capital-gain is < 57 THEN income=>50K 
IF marital-status is Married-civ-spouse AND gender is Female,Male AND capital-gain is < 57 AND capital-loss is 1820.5 - 1859 THEN income=>50K 
IF relationship is Unmarried AND capital-gain is 4826 - 5119 AND capital-loss is < 1551.5 AND native-country is United-States THEN income=>50K 
8197
Blackbox and our, acc 0.8928498305161863
Blackbox and our, f1 score 0.7330839850391023
Blackbox and our,recall 0.6845168800931315
Blackbox and our,precision 0.7890691716481639
number of rules 7
ave number of conditions 2.2857142857142856
m

In [ ]:
from tqdm import tqdm_notebook
%load_ext autoreload
%autoreload 2

import Orange

disc = Orange.preprocess.Discretize()
disc.method = Orange.preprocess.discretize.EntropyMDL()
# disc.method = Orange.preprocess.discretize.EqualFreq(n=3)

disc_predicted_data_table = disc(predicted_data_table)
# disc_predicted_test_data_table = disc(predicted_test_data_table)
disc_predicted_test_data_table = Orange.data.Table.from_table(disc_predicted_data_table.domain, predicted_test_data_table)


from utils import uniform_enlarge_dataset,estimated_enlarge_dataset

rate = 1.0 * 49804 / predicted_data_table.X.shape[0] 
print("sampling rate",rate)
new_predicted_data_table = estimated_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)
print(new_predicted_data_table.X.shape)
disc_new_predicted_data_table =  Orange.data.Table.from_table(disc_predicted_data_table.domain, new_predicted_data_table)

from approach import explain_tabular
explanations,explainer = explain_tabular(disc_new_predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed,beta = 0,use_pre_mined=True, objective = 'bayesian')

from utils import  rule_to_string,ruleset_predict
our_prediction = ruleset_predict(explanations,disc_predicted_test_data_table.X)

import sklearn
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(disc_predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(disc_predicted_test_data_table.Y, our_prediction))

from competition_methods_explanation.passive_methods.brs import compute_metrics_brs
compute_metrics_brs(explanations)

# from competition_methods_explanation.passive_methods.brs import explain_tabular
# explanations = explain_tabular(disc_new_predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
sampling rate 1.1330163568942375
(93761, 12)
init transformer okay!
new cached!


using pre-mined rules
start Generating rule space.  Min support: 0.01
number of premined rules 42385
Screening rules using information gain


### SBRL

In [58]:
from tqdm import tqdm_notebook
%load_ext autoreload
%autoreload 2

import Orange
disc = Orange.preprocess.Discretize()
disc.method = Orange.preprocess.discretize.EntropyMDL()
disc_predicted_test_data_table = disc(predicted_test_data_table)

disc = Orange.preprocess.Discretize()
disc.method = Orange.preprocess.discretize.EntropyMDL()

disc_predicted_data_table = disc(predicted_data_table)
# disc_predicted_test_data_table = disc(predicted_test_data_table)
disc_predicted_test_data_table = Orange.data.Table.from_table(disc_predicted_data_table.domain, predicted_test_data_table)


from utils import uniform_enlarge_dataset,estimated_enlarge_dataset

from competition_methods_explanation.passive_methods.brl import explain_tabular


from utils import ruleset_predict
from copy import deepcopy
import sklearn
sampling_rate_candidates = [0,0.5,1,1.5,2]

rate = 1.0 * 49804 / predicted_data_table.X.shape[0] 
print("sampling rate",rate)
# new_predicted_data_table = uniform_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)
new_predicted_data_table = estimated_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)
        
disc_new_predicted_data_table =  Orange.data.Table.from_table(disc_predicted_data_table.domain, new_predicted_data_table)

sbrl = explain_tabular(disc_new_predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed)

sbrl_prediction = sbrl.predict( (disc_predicted_test_data_table.X).astype(int)  )

f1_score = sklearn.metrics.f1_score(predicted_test_data_table.Y, sbrl_prediction)
acc_score=sklearn.metrics.accuracy_score(predicted_test_data_table.Y, sbrl_prediction)
rec_score = sklearn.metrics.recall_score(predicted_test_data_table.Y, sbrl_prediction)
pre_score = sklearn.metrics.precision_score(predicted_test_data_table.Y, sbrl_prediction)
num = sbrl.n_rules
num_of_instance = rate * predicted_data_table.X.shape[0]

print("metrics",num,f1_score,acc_score,rec_score,pre_score,num_of_instance)

from competition_methods_explanation.passive_methods.brl import compute_metrics_sbrl
compute_metrics_sbrl(sbrl.rule_list)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
sampling rate 1.1330163568942375
first discretize data since SBRL only handles categorical data
initialize okay
start finding rule list
finished finding rule list
metrics 143 0.76200101061142 0.903582395087001 0.7208413001912046 0.8081457663451233 49804.0
number of rules 143
ave number of conditions 1.8531468531468531
max number of conditions 2
used features 12


In [60]:
from tqdm import tqdm_notebook
%load_ext autoreload
%autoreload 2

import Orange
disc = Orange.preprocess.Discretize()
disc.method = Orange.preprocess.discretize.EntropyMDL()
disc_predicted_test_data_table = disc(predicted_test_data_table)

disc = Orange.preprocess.Discretize()
disc.method = Orange.preprocess.discretize.EntropyMDL()

disc_predicted_data_table = disc(predicted_data_table)
# disc_predicted_test_data_table = disc(predicted_test_data_table)
disc_predicted_test_data_table = Orange.data.Table.from_table(disc_predicted_data_table.domain, predicted_test_data_table)


from utils import uniform_enlarge_dataset,estimated_enlarge_dataset

from competition_methods_explanation.passive_methods.brl import explain_tabular


from utils import ruleset_predict
from copy import deepcopy
import sklearn

sampling_rate_candidates = [0,0.5,1,1.5,2]

rate = 0
print("sampling rate",rate)
# new_predicted_data_table = uniform_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)
new_predicted_data_table = estimated_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)
        
disc_new_predicted_data_table =  Orange.data.Table.from_table(disc_predicted_data_table.domain, new_predicted_data_table)

sbrl = explain_tabular(disc_new_predicted_data_table, black_box, target_class_idx=1, random_seed=random_seed)

sbrl_prediction = sbrl.predict( (disc_predicted_test_data_table.X).astype(int)  )

f1_score = sklearn.metrics.f1_score(predicted_test_data_table.Y, sbrl_prediction)
acc_score=sklearn.metrics.accuracy_score(predicted_test_data_table.Y, sbrl_prediction)
rec_score = sklearn.metrics.recall_score(predicted_test_data_table.Y, sbrl_prediction)
pre_score = sklearn.metrics.precision_score(predicted_test_data_table.Y, sbrl_prediction)
num = sbrl.n_rules
num_of_instance = rate * predicted_data_table.X.shape[0]


print("metrics",num,f1_score,acc_score,rec_score,pre_score,num_of_instance)
from competition_methods_explanation.passive_methods.brl import compute_metrics_sbrl
compute_metrics_sbrl(sbrl.rule_list)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
sampling rate 0
first discretize data since SBRL only handles categorical data
initialize okay
start finding rule list
finished finding rule list
metrics 94 0.7588582677165354 0.8996929375639714 0.737093690248566 0.781947261663286 0
number of rules 94
ave number of conditions 1.702127659574468
max number of conditions 2
used features 12


### CART

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import  RandomForestClassifier
# c_passive = MLPClassifier(solver='adam', alpha=1e-5,  hidden_layer_sizes=(100, 100,50,50,10), random_state=random_seed,verbose=False)
# c_passive =RandomForestClassifier(n_estimators=300, n_jobs=5)
# c_passive= SVC(gamma='scale')
# c_passive= DecisionTreeClassifier(max_depth=10)
c_passive= DecisionTreeClassifier(random_state=random_seed,max_leaf_nodes=133)


# from sklearn.compose import make_column_transformer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder,Normalizer
# categorical_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_discrete]
# encoder = make_column_transformer( ( OneHotEncoder(categories='auto',sparse=False),categorical_features_idx),
#                             remainder = 'passthrough'
#                             )
# encoder.fit(predicted_data_table.X)
# c_passive.fit(encoder.transform(predicted_data_table.X), predicted_data_table.Y)
# black_prediction = c_passive.predict(encoder.transform(predicted_test_data_table.X))

c_passive.fit(predicted_data_table.X, predicted_data_table.Y)
black_prediction = c_passive.predict(predicted_test_data_table.X)

print(sum(predicted_test_data_table.Y) )

import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))


1046.0
traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.7573604060913706
Blackbox and our, acc 0.9021494370522006
Blackbox and our,recall 0.7131931166347992
Blackbox and our,precision 0.8073593073593074


In [8]:
estimator = c_passive
n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth

# from collections import defaultdict
# defaultdict(list)
parents = {}
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
        parents[ children_left[node_id]  ] = node_id
        parents[ children_right[node_id]  ] = node_id
    else:
        is_leaves[node_id] = True

# print( len(parents) )
        
print("number of leaves",sum(is_leaves) )

leave_nodes = [ i for i in range( len(is_leaves) ) if is_leaves[i]==True ] 

paths_of_leave_nodes = {}
for i in range(n_nodes):
    if is_leaves[i] == True:
        path = []
        tmp = i
        while( tmp >=0 and tmp in parents) :
            tmp_parent = parents[tmp]
            path.append(tmp_parent)
            tmp = tmp_parent
            
        paths_of_leave_nodes[i] = path

print("average number of conditions", sum ( [   len(set(  feature[n]  for n in paths_of_leave_nodes[i]     ))    for i in leave_nodes] ) / len(leave_nodes)  )
print("max number of conditions", max([   len(set(  feature[n]  for n in paths_of_leave_nodes[i]     ))    for i in leave_nodes] )  )

features_used = set ( [feature[i] for i in range(n_nodes)  if is_leaves[i]!=True ]  ) # the feature array does not use any feature split
print("feature used", len(features_used))


number of leaves 133
average number of conditions 6.548872180451128
max number of conditions 10
feature used 10


In [132]:
from sklearn.tree import DecisionTreeClassifier
c_passive= DecisionTreeClassifier(random_state=random_seed)

from utils import uniform_enlarge_dataset,estimated_enlarge_dataset
rate = ADS_active.synthetic_data_table.X.shape[0] / predicted_data_table.X.shape[0]
print("sampling rate",rate)
# new_predicted_data_table = uniform_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)
new_predicted_data_table = estimated_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)


c_passive.fit(new_predicted_data_table.X, new_predicted_data_table.Y)
black_prediction = c_passive.predict(predicted_test_data_table.X)



import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))

estimator = c_passive
n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth

# from collections import defaultdict
# defaultdict(list)
parents = {}
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
        parents[ children_left[node_id]  ] = node_id
        parents[ children_right[node_id]  ] = node_id
    else:
        is_leaves[node_id] = True

# print( len(parents) )
        
print("number of leaves",sum(is_leaves) )

leave_nodes = [ i for i in range( len(is_leaves) ) if is_leaves[i]==True ] 

paths_of_leave_nodes = {}
for i in range(n_nodes):
    if is_leaves[i] == True:
        path = []
        tmp = i
        while( tmp >=0 and tmp in parents) :
            tmp_parent = parents[tmp]
            path.append(tmp_parent)
            tmp = tmp_parent
            
        paths_of_leave_nodes[i] = path

print("average number of conditions", sum ( [   len(set(  feature[n]  for n in paths_of_leave_nodes[i]     ))    for i in leave_nodes] ) / len(leave_nodes)  )
print("max number of conditions", max([   len(set(  feature[n]  for n in paths_of_leave_nodes[i]     ))    for i in leave_nodes] )  )

features_used = set ( [feature[i] for i in range(n_nodes)  if is_leaves[i]!=True ]  ) # the feature array does not use any feature split
print("feature used", len(features_used))


sampling rate 1.116636713151489
traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.7967939651107968
Blackbox and our, acc 0.9117707267144319
Blackbox and our,recall 0.8078393881453155
Blackbox and our,precision 0.786046511627907
number of leaves 7741
average number of conditions 8.66464281100633
max number of conditions 12
feature used 12


### Random forest

In [139]:
from sklearn.ensemble import  RandomForestClassifier
# c_passive = MLPClassifier(solver='adam', alpha=1e-5,  hidden_layer_sizes=(100, 100,50,50,10), random_state=random_seed,verbose=False)
c_passive =RandomForestClassifier(random_state=random_seed)
# c_passive= SVC(gamma='scale')
# c_passive= DecisionTreeClassifier(max_depth=10)
# c_passive= DecisionTreeClassifier(random_state=random_seed)


# from sklearn.compose import make_column_transformer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder,Normalizer
# categorical_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_discrete]
# encoder = make_column_transformer( ( OneHotEncoder(categories='auto',sparse=False),categorical_features_idx),
#                             remainder = 'passthrough'
#                             )
# encoder.fit(predicted_data_table.X)
# c_passive.fit(encoder.transform(predicted_data_table.X), predicted_data_table.Y)
# black_prediction = c_passive.predict(encoder.transform(predicted_test_data_table.X))

c_passive.fit(predicted_data_table.X, predicted_data_table.Y)
black_prediction = c_passive.predict(predicted_test_data_table.X)

print(sum(predicted_test_data_table.Y) )

import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))



1046.0
traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.8242485990830362
Blackbox and our, acc 0.9293756397134084
Blackbox and our,recall 0.7734225621414914
Blackbox and our,precision 0.8822246455834242


In [142]:


def get_path_from_tree(decision_tree):
    
    

    n_nodes = decision_tree.tree_.node_count
    children_left = decision_tree.tree_.children_left
    children_right = decision_tree.tree_.children_right
    feature = decision_tree.tree_.feature
    threshold = decision_tree.tree_.threshold

    # The tree structure can be traversed to compute various properties such
    # as the depth of each node and whether or not it is a leaf.
    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]  # seed is the root node id and its parent depth

    parents = {}
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1

        # If we have a test node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
            parents[ children_left[node_id]  ] = node_id
            parents[ children_right[node_id]  ] = node_id
        else:
            is_leaves[node_id] = True


    leave_nodes = [ i for i in range( len(is_leaves) ) if is_leaves[i]==True ] 

    paths_of_leave_nodes = {}
    for i in range(n_nodes):
        if is_leaves[i] == True:
            path = []
            tmp = i
            while( tmp >=0 and tmp in parents) :
                tmp_parent = parents[tmp]
                path.append(tmp_parent)
                tmp = tmp_parent

            paths_of_leave_nodes[i] = path
    return [   set(  feature[n]  for n in paths_of_leave_nodes[i]  ) for i in leave_nodes]

list_of_paths = [ get_path_from_tree(t) for t in c_passive.estimators_ ]
import itertools
all_paths = list(itertools.chain.from_iterable(list_of_paths))


print("number of all leaves", len(all_paths))
print("average number of conditions", sum ( [   len(i)    for i in all_paths] ) / len(all_paths)  )
print("max number of conditions", max ( [   len(i)    for i in all_paths] ) )



number of all leaves 44687
average number of conditions 9.571351847293396
max number of conditions 12


In [143]:
from sklearn.ensemble import  RandomForestClassifier

c_passive =RandomForestClassifier(random_state=random_seed)


from utils import uniform_enlarge_dataset,estimated_enlarge_dataset
rate = ADS_active.synthetic_data_table.X.shape[0] / predicted_data_table.X.shape[0]
print("sampling rate",rate)
new_predicted_data_table = estimated_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)


c_passive.fit(new_predicted_data_table.X, new_predicted_data_table.Y)
black_prediction = c_passive.predict(predicted_test_data_table.X)





print(sum(predicted_test_data_table.Y) )

import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))




def get_path_from_tree(decision_tree):
    
    

    n_nodes = decision_tree.tree_.node_count
    children_left = decision_tree.tree_.children_left
    children_right = decision_tree.tree_.children_right
    feature = decision_tree.tree_.feature
    threshold = decision_tree.tree_.threshold

    # The tree structure can be traversed to compute various properties such
    # as the depth of each node and whether or not it is a leaf.
    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]  # seed is the root node id and its parent depth

    parents = {}
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth[node_id] = parent_depth + 1

        # If we have a test node
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
            parents[ children_left[node_id]  ] = node_id
            parents[ children_right[node_id]  ] = node_id
        else:
            is_leaves[node_id] = True


    leave_nodes = [ i for i in range( len(is_leaves) ) if is_leaves[i]==True ] 

    paths_of_leave_nodes = {}
    for i in range(n_nodes):
        if is_leaves[i] == True:
            path = []
            tmp = i
            while( tmp >=0 and tmp in parents) :
                tmp_parent = parents[tmp]
                path.append(tmp_parent)
                tmp = tmp_parent

            paths_of_leave_nodes[i] = path
    return [   set(  feature[n]  for n in paths_of_leave_nodes[i]  ) for i in leave_nodes]

list_of_paths = [ get_path_from_tree(t) for t in c_passive.estimators_ ]
import itertools
all_paths = list(itertools.chain.from_iterable(list_of_paths))


print("number of all leaves", len(all_paths))
print("average number of conditions", sum ( [   len(i)    for i in all_paths] ) / len(all_paths)  )
print("max number of conditions", max ( [   len(i)    for i in all_paths] ) )

sampling rate 1.116636713151489
1046.0
traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.8310362429811129
Blackbox and our, acc 0.9322415557830092
Blackbox and our,recall 0.7782026768642447
Blackbox and our,precision 0.891566265060241
number of all leaves 82850
average number of conditions 9.32267954133977
max number of conditions 12


### SVM

In [155]:
from sklearn.ensemble import  RandomForestClassifier
from sklearn.svm import SVC
c_passive = SVC(gamma='scale',random_state=random_seed)

# c_passive.fit(predicted_data_table.X, predicted_data_table.Y)
# black_prediction = c_passive.predict(predicted_test_data_table.X)


from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
categorical_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_discrete]
continuous_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_continuous]

scikit_encoder = make_column_transformer( ( OneHotEncoder(categories='auto',sparse=False),categorical_features_idx),
(StandardScaler(), continuous_features_idx),
                    remainder = 'passthrough'
                    )

scikit_encoder.fit(data_table.X)

c_passive.fit(scikit_encoder.transform(predicted_data_table.X), predicted_data_table.Y)
black_prediction = c_passive.predict( scikit_encoder.transform(predicted_test_data_table.X ))

import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))
print("number of support vectors",c_passive.support_vectors_.shape)

traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.7884231536926146
Blackbox and our, acc 0.9132036847492323
Blackbox and our,recall 0.7552581261950286
Blackbox and our,precision 0.824634655532359
number of support vectors (10866, 103)


In [160]:
c_passive = SVC(gamma='scale',random_state=random_seed)

from utils import uniform_enlarge_dataset,estimated_enlarge_dataset
rate = ADS_active.synthetic_data_table.X.shape[0] / predicted_data_table.X.shape[0]
print("sampling rate",rate)
new_predicted_data_table = estimated_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)



from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
categorical_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_discrete]
continuous_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_continuous]

scikit_encoder = make_column_transformer( ( OneHotEncoder(categories='auto',sparse=False),categorical_features_idx),
(StandardScaler(), continuous_features_idx),
                    remainder = 'passthrough'
                    )

scikit_encoder.fit(data_table.X)


c_passive.fit(scikit_encoder.transform(new_predicted_data_table.X), new_predicted_data_table.Y)
black_prediction = c_passive.predict( scikit_encoder.transform(predicted_test_data_table.X ))


import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))
print("number of support vectors",c_passive.support_vectors_.shape)

sampling rate 1.116636713151489
traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.7918987341772152
Blackbox and our, acc 0.9158648925281474
Blackbox and our,recall 0.7476099426386233
Blackbox and our,precision 0.8417653390742734
number of support vectors (22689, 103)


### DNN

In [167]:
from sklearn.neural_network import MLPClassifier
c_passive = MLPClassifier(solver='adam', alpha=1e-3,  hidden_layer_sizes=(100,100,50,50,10), random_state=random_seed,verbose=False)

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
categorical_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_discrete]
continuous_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_continuous]

scikit_encoder = make_column_transformer( ( OneHotEncoder(categories='auto',sparse=False),categorical_features_idx),
(StandardScaler(), continuous_features_idx),
                    remainder = 'passthrough'
                    )

scikit_encoder.fit(data_table.X)

c_passive.fit(scikit_encoder.transform(predicted_data_table.X), predicted_data_table.Y)
black_prediction = c_passive.predict( scikit_encoder.transform(predicted_test_data_table.X ))

import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))
print("number of parameter", sum([  layer.shape[0]*layer.shape[1]  for layer in c_passive.coefs_ ]))

traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.854079254079254
Blackbox and our, acc 0.9359263050153531
Blackbox and our,recall 0.875717017208413
Blackbox and our,precision 0.8334849863512284
number of parameter 28310


In [168]:
from sklearn.neural_network import MLPClassifier
c_passive = MLPClassifier(solver='adam', alpha=1e-3,  hidden_layer_sizes=(100,100,50,50,10), random_state=random_seed,verbose=False)


from utils import uniform_enlarge_dataset,estimated_enlarge_dataset
rate = ADS_active.synthetic_data_table.X.shape[0] / predicted_data_table.X.shape[0]
print("sampling rate",rate)
new_predicted_data_table = estimated_enlarge_dataset(predicted_data_table,black_box,sampling_rate=rate,random_seed=random_seed)



from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
categorical_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_discrete]
continuous_features_idx = [i for i,a in enumerate(data_table.domain.attributes) if a.is_continuous]

scikit_encoder = make_column_transformer( ( OneHotEncoder(categories='auto',sparse=False),categorical_features_idx),
(StandardScaler(), continuous_features_idx),
                    remainder = 'passthrough'
                    )

scikit_encoder.fit(data_table.X)


c_passive.fit(scikit_encoder.transform(new_predicted_data_table.X), new_predicted_data_table.Y)
black_prediction = c_passive.predict( scikit_encoder.transform(predicted_test_data_table.X ))


import sklearn
print("traning a blackbox to approximate a blackbox, with original dataset")
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, black_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, black_prediction))
print("number of parameter", sum([  layer.shape[0]*layer.shape[1]  for layer in c_passive.coefs_ ]))

sampling rate 1.116636713151489
traning a blackbox to approximate a blackbox, with original dataset
Blackbox and our, f1 score 0.8691099476439792
Blackbox and our, acc 0.9437052200614124
Blackbox and our,recall 0.872848948374761
Blackbox and our,precision 0.8654028436018958
number of parameter 28310


## Active baselines

### DTExtract

In [9]:
a = np.array([ [-1.7, -1.5, -0.2, 0.2, 1.5, 1.7, 2.3],[-1.7, -1.5, -0.2, 0.2, 1.5, 1.7, 2.3] ])

tmp = np.rint(a,where=[[0,1,0,0,0,0,0], [1,0,0,0,0,0,0]] )
print(tmp)

[[ 0. -2.  0. nan nan nan nan]
 [-2. nan nan nan nan nan nan]]


In [10]:
%load_ext autoreload
%autoreload 2
from competition_methods_explanation.active_methods_top_down.dt import explain_tabular
print(predicted_data_table.domain)
explanations, tree = explain_tabular(
    predicted_data_table,  black_box, target_class_idx=1, random_seed=42, active_instance_number = 0,termination_max=100)


from utils import  rule_to_string,ruleset_predict
# for e in explanations:
#     print(rule_to_string(e,predicted_data_table.domain,target_class_idx=1))
#     try:
#         print(rule_to_string(e,disc_predicted_data_table.domain,target_class_idx=1))
#     except:
#         continue
        

our_prediction = ruleset_predict(explanations,predicted_test_data_table.X)
tree_our_prediction = tree.predict(predicted_test_data_table.X)

print(sum(our_prediction) )
print(sum(tree_our_prediction))

our_prediction = tree_our_prediction

import sklearn
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

from competition_methods_explanation.active_methods_top_down.dt import compute_metrics_dt
compute_metrics_dt(explanations)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[age, workclass, education, marital-status, occupation, relationship, race, gender, capital-gain, capital-loss, hours-per-week, native-country | income]



400
400.0
Blackbox and our, acc 0.8509723643807574
Blackbox and our, f1 score 0.4965421853388658
Blackbox and our,recall 0.34321223709369025
Blackbox and our,precision 0.8975
number of rules 45
ave number of conditions 6.0
max number of conditions 9
used features 10


In [23]:
%load_ext autoreload
%autoreload 2
from competition_methods_explanation.active_methods_top_down.dt import explain_tabular

explanations, tree = explain_tabular(
    predicted_data_table,  black_box, target_class_idx=1, random_seed=42, active_instance_number = 2000,termination_max=100)


from utils import  rule_to_string,ruleset_predict
# for e in explanations:
#     print(rule_to_string(e,predicted_data_table.domain,target_class_idx=1))
#     try:
#         print(rule_to_string(e,disc_predicted_data_table.domain,target_class_idx=1))
#     except:
#         continue
        
our_prediction = ruleset_predict(explanations,predicted_test_data_table.X)
tree_our_prediction = tree.predict(predicted_test_data_table.X)

print( sum(our_prediction) )
print(sum(tree_our_prediction))

our_prediction = tree_our_prediction

import sklearn
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

from competition_methods_explanation.active_methods_top_down.dt import compute_metrics_dt
compute_metrics_dt(explanations)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
start estimate the distribution
using gaussian distribution
finish estimate the distribution


266
266.0
Blackbox and our, acc 0.8325486182190379
Blackbox and our, f1 score 0.3765243902439025
Blackbox and our,recall 0.2361376673040153
Blackbox and our,precision 0.9285714285714286
number of rules 49
ave number of conditions 6.26530612244898
max number of conditions 9
used features 11


### Anchor global version

In [46]:
%load_ext autoreload
%autoreload 2
from competition_methods_explanation.active_methods_bottom_up.cn2anchor import cn2anchor_tabular
explanations = cn2anchor_tabular(predicted_data_table, black_box, target_class_idx = 1, random_seed=42)

from utils import  rule_to_string,ruleset_predict
# for e in explanations:
#     print(rule_to_string(e,predicted_data_table.domain,target_class_idx=1))
#     try:
#         print(rule_to_string(e,disc_predicted_data_table.domain,target_class_idx=1))
#     except:
#         continue
        
our_prediction = ruleset_predict(explanations,predicted_test_data_table.X)

print( sum(our_prediction) )
import sklearn
print('Blackbox and our, acc', sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our, f1 score', sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,recall', sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction))
print('Blackbox and our,precision', sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction))

# from competition_methods_explanation.active_methods_top_down.dt import compute_metrics_dt
from utils import compute_metrics
compute_metrics(explanations,predicted_data_table.domain)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


KeyboardInterrupt: 

# Parameter Tuning


tuing two parameter is relatively easy, we loop $\lambda$ and $\beta$ over a range of values and keep record of the resulted faithfulness and interpretability


In [ ]:
from tqdm import tqdm_notebook
%load_ext autoreload
%autoreload 2
from approach import explain_tabular
from utils import ruleset_predict
from copy import deepcopy

lambda_candidates = [  0,0.0000001,0.000001,0.00001,0.00003,0.00005,0.00008,0.0001,0.0005,0.001,0.0013,0.0015,0.0018,0.002,0.0025,0.003,0.005,0.006,0.007,0.008,0.01,0.015,0.02,0.03,0.05,0.1,0.15,0.2,0.5 ]

beta_candidates = [0,0.2,0.5,1.5,2,3,5,10]
beta_candidates = [i * 0.00001 for i in beta_candidates]

# lambda_candidates = [  0.01 ]
# beta_candidates = [0]
results={}
for beta in beta_candidates:
    results[beta] = []


for beta in tqdm_notebook(beta_candidates): 
    lambda_parameter = lambda_candidates[0]
    explanations,ADS = explain_tabular(predicted_data_table, black_box, target_class_idx=1, random_seed=42,beta=beta,lambda_parameter=lambda_parameter)
    for lambda_parameter in  lambda_candidates:
        explanations = ADS.output_the_best(lambda_parameter)
        our_prediction = ruleset_predict(explanations,test_data_table.X)
        f1_score = sklearn.metrics.f1_score(predicted_test_data_table.Y, our_prediction)
        acc_score=sklearn.metrics.accuracy_score(predicted_test_data_table.Y, our_prediction)
        rec_score = sklearn.metrics.recall_score(predicted_test_data_table.Y, our_prediction)
        pre_score = sklearn.metrics.precision_score(predicted_test_data_table.Y, our_prediction)
        num = len(explanations)
        num_of_instance = ADS.synthetic_data_table.X.shape[0]
        results[beta].append( (lambda_parameter,(num,f1_score,acc_score,rec_score,pre_score,num_of_instance,deepcopy(explanations) )  ) )
        print("parameter: ",beta,lambda_parameter,"metrics",num,f1_score,acc_score,rec_score,pre_score,num_of_instance)



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import math
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
# name = "Greys"
# cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
# colors = cmap  # type: list

# # plt.set_prop_cycle(color=colors)

# color_map = get_cmap('viridis')
color_map = get_cmap('binary')
# color_map = get_cmap('winter')
beta_candidates = [0,0.2,0.5,1.5,2,3,5,10]
beta_candidates = [i * 0.00001 for i in beta_candidates]


for count,b in enumerate(beta_candidates):
#     if count in [3]: 
#         continue
    plot_use_x = []
    plot_use_y = []
    for i in range(len(lambda_candidates)):
        lambda_parameter,(num,f1_score,acc_score,rec_score,pre_score,num_of_instance,explanations) = results[b][i]
        plot_use_x.append(num)
#         plot_use_y.append(f1_score)
        plot_use_y.append(acc_score)
        
    # for x,y in zip(plot_use_x,plot_use_y):
    #     print(x,y)
    tmp = [(x,y) for x,y in zip(plot_use_x,plot_use_y) ]
#     tmp.append((0,0.5))
#     tmp.append((100,1))
    tmp = sorted(tmp,key=lambda x:x[0])
    X =np.asarray([ t[0] for t in tmp]) .reshape(-1, 1)
    y =np.asarray([ t[1] for t in tmp]) .reshape(-1, 1)
#     clf = GaussianProcessRegressor()
#     clf.fit(X, y) 
#     x_virtul = [ 0.1 * i for i in range(50)]
#     x_for_predict = np.asarray(x_virtul) .reshape(-1, 1)
#     plt.plot(x_virtul,clf.predict(x_for_predict).reshape(-1),color=color_map(  math.sqrt(b*1000)  )  )
#     plt.plot(X,y  )
    plt.plot(X,y,color=color_map(  math.sqrt(b*5000)  )  )

#     format(12.456789, '.3e')
plt.legend([str(b)for b in beta_candidates],loc=(1.04,0))
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# plt.legend([str(b) for b in beta_candidates], loc=5)
# plt.xlim([3,10])
# plt.ylim([0.8,1])
plt.ylabel('f1 score')
plt.xlabel('number of rules')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import math
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
# name = "Greys"
# cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
# colors = cmap  # type: list

# # plt.set_prop_cycle(color=colors)

# color_map = get_cmap('viridis')
color_map = get_cmap('binary')



# lambda_candidates = [  0,0.0000001,0.000001,0.00001,0.00003,0.00005,0.00008,0.0001,0.0005,0.001,0.0013,0.0015,0.0018,0.002,0.0025,0.003,0.005,0.006,0.007,0.008,0.01,0.015,0.02,0.03,0.05,0.1,0.15,0.2,0.5 ]


# beta_candidates = [0,0.2,0.5,1,1.5,2,3,5,10]
# beta_candidates = [i * 0.00001 for i in beta_candidates]

for b in beta_candidates:
    plot_use_x = []
    plot_use_y = []
    for i in range(len(lambda_candidates)):
        lambda_parameter,(num,f1_score,acc_score,rec_score,pre_score,num_of_instance,explanations) = results[b][i]
        plot_use_x.append(lambda_parameter)
        plot_use_y.append(f1_score)
    # for x,y in zip(plot_use_x,plot_use_y):
    #     print(x,y)
   
    plt.plot(plot_use_x,plot_use_y,color=color_map(  math.sqrt(b*5000)  ))
plt.legend([str(b) for b in beta_candidates], loc=(1.04,0))

plt.ylabel('f1 score')
plt.xlabel('lambda')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

color_map = get_cmap('binary')
for b in beta_candidates:
    plot_use_x = []
    plot_use_y = []
    for i in range(len(lambda_candidates)):
        lambda_parameter,(num,f1_score,acc_score,rec_score,pre_score,num_of_instance,explanations) = results[b][i]
        plot_use_x.append(lambda_parameter)
        plot_use_y.append(num)
    # for x,y in zip(plot_use_x,plot_use_y):
    #     print(x,y)
    plt.plot(plot_use_x,plot_use_y,color=color_map(  math.sqrt(b*1000)  ))
plt.legend([str(b) for b in beta_candidates], loc=(1.04,0))

plt.ylabel('num of rules')
plt.xlabel('lambda')
plt.xlim([0,0.01])
plt.show()

In [ ]:
score_list=[]
for i,l in enumerate(lambda_candidates):
    for b in beta_candidates:

        lambda_parameter,(num,f1_score,acc_score,rec_score,pre_score,num_of_instance,explanations) = results[b][i]
        score_list.append(  (l,b,num,f1_score,acc_score,rec_score,pre_score) )

score_list = sorted(score_list,reverse=True,key=lambda x:x[4])
print(score_list[0])
l,b,num,f1_score,acc_score,rec_score,pre_score = score_list[0]
print("the best lambda: ",l)
print("the best beta:",b)
print("f1:",f1_score," acc: ",acc_score," recall: ",rec_score," precision: ",pre_score)

In [ ]:
for b in results:
    print(b)

In [ ]:
from pprint import pprint as pr
pr(results)